# One Stop Stock Analyser

A basic and fundamental stock analyser for my personal stock research usage.  
Bug, equation errors is inevitable. 

Available Information:
1. Company Background
2. Profitability, Financial Health, and Growth
3. Altman Z-Score, Beneish M-Score, WACC, ROIC, Peter Lynch Fair Value
4. Discounted Free Cash Flow (DFCF) Model
5. Historical Price, SMA Signal

In [ ]:
import yfinance as yf
import datetime
import pandas as pd
from PIL import Image
import requests
import matplotlib.pyplot as plt
import numpy as np
import requests
from bs4 import BeautifulSoup

### Input your ticker symbol here

In [ ]:
ticker = input()
ticker = ticker.upper()
stock = yf.Ticker(ticker)

logo_url = stock.info['logo_url']
Image.open(requests.get(logo_url, stream=True).raw)

## Company Summary

In [ ]:
company_summary = stock.info['longBusinessSummary']
print(company_summary, '\n')

website_url = stock.info['website']
print('Website: {} \n'.format(website_url))

sector = stock.info['sector']
print('Sector: {} \n'.format(sector))

industry = stock.info['industry']
print('Industry: {} \n'.format(industry))

market_capital = stock.info['marketCap']
print('Market Capital: {} Billions \n'.format(market_capital / 1000000000))

low52 = stock.info['fiftyTwoWeekLow']
print('52-Low: {} \n'.format(low52))

high52 = stock.info['fiftyTwoWeekHigh']
print('52-High: {} \n'.format(high52))

current_price = stock.info['currentPrice']
print('Current Price: {} \n'.format(round(current_price, 2)))

down_from_ath = ((current_price / high52) - 1) * 100
print('Down From ATH: {}% \n'.format(round(down_from_ath, 2)))

peg_ratio = stock.info['pegRatio']
print('Peg Ratio: {} \n'.format(round(peg_ratio, 2)))

price_to_earning_ratio = stock.info['trailingPE']
print('P/E Ratio: {} \n'.format(round(price_to_earning_ratio, 2)))

price_to_sales_ratio = stock.info['priceToSalesTrailing12Months']
print('P/S Ratio: {} \n'.format(round(price_to_sales_ratio, 2)))

## Profitability

In [ ]:
gross_margins = stock.info['grossMargins']
print('Gross Margins: {}% \n'.format( round(gross_margins * 100, 2)))

profit_margins = stock.info['profitMargins']
print('Profit Margins: {}% \n'.format( round(profit_margins * 100, 2)))

operating_margins = stock.info['operatingMargins']
print('Operating Margins: {}% \n'.format( round(operating_margins * 100, 2)))

return_on_assets = stock.info['returnOnAssets']
print('Return On Assets: {}% \n'.format( round(return_on_assets * 100, 2)))

return_on_equity = stock.info['returnOnEquity']
print('Return On Equity: {}% \n'.format( round(return_on_equity * 100, 2)))

ebitda_margin= stock.info['ebitda'] / stock.info['totalRevenue']
print('Ebitda Margin: {}% \n'.format( round(ebitda_margin * 100, 2)))

## Financial Health

In [ ]:
quick_ratio = stock.info['quickRatio']
print('Quick Ratio -> (Total Asset - Inventory) / Total Liability')
print('Quick Ratio: {} \n'.format( round(quick_ratio, 2)))

debt_to_equity_ratio = stock.info['debtToEquity'] / 100
print('Debt to Equity Ratio -> How much debt a company is using to finance its assets relative to the value of shareholders\' equity.')
print('Debt to Equity Ratio: {} \n'.format( round(debt_to_equity_ratio, 2)))

balance_sheet = stock.get_balancesheet(freq='quarterly')
balance_sheet = balance_sheet.fillna(0)
long_term_debt = balance_sheet.T['Long Term Debt'][0]
total_equity = balance_sheet.T['Total Stockholder Equity'][0] + balance_sheet.T['Other Stockholder Equity'][0]
long_term_debt_to_equity_ratio = long_term_debt / total_equity
print('Long Term Debt to Equity Ratio: {} \n'.format( round(long_term_debt_to_equity_ratio, 2)))

## Growth

In [ ]:
revenue_growth = stock.info['revenueGrowth']
print('Revenue Growth: {}% \n'.format( round(revenue_growth * 100, 2)))

earnings_growth = stock.info['earningsQuarterlyGrowth']
print('Earnings Growth: {}% \n'.format( round(earnings_growth * 100, 2)))

## Altman Z-Score/Credit Strength Test

In [ ]:
balance_sheet = stock.get_balancesheet(freq='quarterly')
balance_sheet = balance_sheet.fillna(0)
income_statement = stock.get_financials(freq='yearly')
income_statement = income_statement.fillna(0)
total_assets = balance_sheet.T['Total Assets'][0]
total_liabilities = balance_sheet.T['Total Liab'][0]

working_capital = balance_sheet.T['Total Current Assets'][0] - balance_sheet.T['Total Current Liabilities'][0]
A1 = working_capital / total_assets

retained_earnings = balance_sheet.T['Retained Earnings'][0] 
B1 = retained_earnings / total_assets

ebit = income_statement.T['Income Before Tax'][0] - income_statement.T['Interest Expense'][0]
C1 = ebit / total_assets

market_capital = stock.info['marketCap']
D1 = market_capital / total_liabilities

# Replace sales with total revenue
E1 = income_statement.T['Total Revenue'][0] / total_assets

altman_zscore = (1.2 * A1) + (1.4 * B1) + (3.3 * C1) + (0.6 * D1) + (1.0 * E1)

ticker_altman_zscore_status = None

if altman_zscore <= 1.8:
    ticker_altman_zscore_status = 'Distress Zone'
elif altman_zscore >= 3:
    ticker_altman_zscore_status = 'Safe Zones'
else:
    ticker_altman_zscore_status = 'Grey Zones'

print('When <= 1.8, Distress Zones')
print('When >= 3, Safe Zones')
print('When between, Grey Zones')

print('\nAltman Z-Score for {} is {}, hence {}'.format(ticker, round(altman_zscore,2), ticker_altman_zscore_status))

## Beneish M-Score/Earnings Manipulation Detection

In [ ]:
balance_sheet = stock.get_balancesheet(freq='yearly')
balance_sheet = balance_sheet.fillna(0)
income_statement = stock.get_financials(freq='yearly')
income_statement = income_statement.fillna(0)
cash_flow_statement = stock.get_cashflow(freq='yearly')
cash_flow_statement = cash_flow_statement.fillna(0)

# Days' Sales in Receivables Index
cy_net_receivables = balance_sheet.T['Net Receivables'][0]
py_net_receivables = balance_sheet.T['Net Receivables'][1]
cy_total_revenue = income_statement.T['Total Revenue'][0]
py_total_revenue = income_statement.T['Total Revenue'][1]
sub_dsri_cy = cy_net_receivables / cy_total_revenue
sub_dsri_py = py_net_receivables / py_total_revenue
dsri = sub_dsri_cy / sub_dsri_py

# Gross Margin Index
cy_cost_of_revenue = income_statement.T['Cost Of Revenue'][0]
py_cost_of_revenue = income_statement.T['Cost Of Revenue'][1]
sub_gmi_py = (py_total_revenue - py_cost_of_revenue) / py_total_revenue
sub_gmi_cy = (cy_total_revenue - cy_cost_of_revenue) / cy_total_revenue
gmi = sub_gmi_py / sub_gmi_cy

# Asset Quality Index
cy_total_current_assets = balance_sheet.T['Total Current Assets'][0]
py_total_current_assets = balance_sheet.T['Total Current Assets'][1]
cy_ppe = balance_sheet.T['Property Plant Equipment'][0]
py_ppe = balance_sheet.T['Property Plant Equipment'][1]
cy_total_assets = balance_sheet.T['Total Assets'][0]
py_total_assets = balance_sheet.T['Total Assets'][1]
sub_aqi_cy = (1 - (cy_total_current_assets + cy_ppe) ) / cy_total_assets
sub_aqi_py = (1 - (py_total_current_assets + py_ppe) ) / py_total_assets
aqi = sub_aqi_cy / sub_aqi_py

# Sales Growth Index
sgi = cy_total_revenue / py_total_revenue

# Depreciation Index
cy_depreciation = cash_flow_statement.T['Depreciation'][0]
py_depreciation = cash_flow_statement.T['Depreciation'][1]
sub_depi_py = py_depreciation / (py_depreciation + py_ppe)
sub_depi_cy = cy_depreciation / (cy_depreciation + cy_ppe)
depi = sub_depi_py / sub_depi_cy

# Sales, General and Administrative Expenses Index
cy_sga = income_statement.T['Selling General Administrative'][0]
py_sga = income_statement.T['Selling General Administrative'][1]
sub_sgai_cy = cy_sga / cy_total_revenue
sub_sgai_py = py_sga / py_total_revenue
sgai = sub_sgai_cy / sub_sgai_py

# Total Accruals to Total Assets Index
cy_total_current_assets = balance_sheet.T['Total Current Assets'][0]
py_total_current_assets = balance_sheet.T['Total Current Assets'][1]
cy_total_current_liabilities = balance_sheet.T['Total Current Liabilities'][0]
py_total_current_liabilities = balance_sheet.T['Total Current Liabilities'][1]
cy_working_capital = cy_total_current_assets - cy_total_current_liabilities
py_working_capital = py_total_current_assets - py_total_current_liabilities
cy_cash = balance_sheet.T['Cash'][0]
py_cash = balance_sheet.T['Cash'][1]
cy_income_tax_expense = income_statement.T['Income Tax Expense'][0]
py_income_tax_expense = income_statement.T['Income Tax Expense'][1]
cy_long_term_debt = balance_sheet.T['Long Term Debt'][0]
py_long_term_debt = balance_sheet.T['Long Term Debt'][1]

sub_tata_1 = (cy_working_capital - py_working_capital) - (cy_cash - py_cash)
sub_tata_2 = cy_income_tax_expense - py_income_tax_expense
sub_tata_3 = cy_long_term_debt - py_long_term_debt
tata = (sub_tata_1 + sub_tata_2 + sub_tata_3 - cy_depreciation) / cy_total_assets

# Leverage Index
cy_total_liabilities = balance_sheet.T['Total Liab'][0]
py_total_liabilities = balance_sheet.T['Total Liab'][1]
sub_lvgi_cy = (cy_long_term_debt + cy_total_current_liabilities) / cy_total_assets
sub_lvgi_py = (py_long_term_debt + py_total_current_liabilities) / py_total_assets
lvgi = sub_lvgi_cy / sub_lvgi_py

beneish_mscore = -4.84 + (0.92*dsri) + (0.528*gmi) + (0.404*aqi) + (0.892*sgi) + (0.115*depi)\
- (0.172*sgai) + (4.679*tata) - (0.327*lvgi)

ticker_beneish_mscore_status = None

if beneish_mscore <= -2.22:
    ticker_beneish_mscore_status = 'No Manipulation Done'
elif beneish_mscore >= -1.78:
    ticker_beneish_mscore_status = 'Likely Manipulation Done'
else:
    ticker_beneish_mscore_status = 'Possible Manipulation Done'

print('When <= -2.22, No Manipulation Done')
print('When >= -1.78, Likely Manipulation Done')
print('When between, Possible Manipulation Done')

print('\nBeneish M-Score for {} is {}, hence {}'.format(ticker, round(beneish_mscore,2), ticker_beneish_mscore_status))

## Weighted Average Cost of Capital

WACC = (E / V * Re ) + (D / V * Rd * (1 - Tc)) <br>

E = Market value of the firm’s equity/ Market Capital <br>
D = Market value of the firm’s debt/ Lt Debt + St Debt <br>
V = E + D <br>
Re = Cost of Equity <br>
Rd = Cost of Debt <br>
Tc = Corporate Tax <br>

In [ ]:
def get_balance_sheet_from_yfinance_web(ticker):
    url = f"https://finance.yahoo.com/quote/{ticker}/balance-sheet?p={ticker}"
    header = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
    }  
    r = requests.get(url, headers=header, timeout=5)
    html = r.text
    soup = BeautifulSoup(html, 'html.parser')

    div = soup.find_all('div', attrs={'class': 'D(tbhg)'})
    if len(div) < 1:
        print('Fail to retrieve table column header')
        exit(0)

    col = []
    for h in div[0].find_all('span'):
        text = h.get_text()
        if text != 'Breakdown':
            col.append( datetime.datetime.strptime(text, '%m/%d/%Y') )
    
    df = pd.DataFrame(columns=col)
    for div in soup.find_all('div', attrs={'data-test': 'fin-row'}):
        i = 0
        idx = ""
        val = []
        for h in div.find_all('span'):
            if i == 0:
                idx = h.get_text()
            else:
                num = int(h.get_text().replace(",", "")) * 1000
                val.append( num )
            i += 1
        if len(val) == 4:
            row = pd.DataFrame([val], columns=col, index=[idx] )
        else:
            required_to_fill = 4 - len(val)
            for _ in range(required_to_fill):
                val.append(0)
            row = pd.DataFrame([val], columns=col, index=[idx] )
        df = df.append(row)

    return df

In [ ]:
ebalance_sheet = get_balance_sheet_from_yfinance_web(ticker)
ebalance_sheet = ebalance_sheet.fillna(0)

# Market value of the firm’s equity/ Market Capital
e1 = stock.info['marketCap'] 

# Market value of the firm’s debt/ Lt Debt + St Debt
try:
    capital_lease_obligations = ebalance_sheet.T['Capital Lease Obligations'][0]
    d1 = balance_sheet.T['Long Term Debt'][0] + capital_lease_obligations + balance_sheet.T['Short Long Term Debt'][0]
except:
    d1 = balance_sheet.T['Long Term Debt'][0] + balance_sheet.T['Short Long Term Debt'][0]

v = e1 + d1

# Cost of Equity
risk_free_rate_of_return = 3.28 / 100 # 10-Year Treasury Constant Maturity Rate refer https://fred.stlouisfed.org/series/DGS10
beta = stock.info['beta']
market_premium = 6 / 100
re1 = (risk_free_rate_of_return + (beta * market_premium) ) * 100

# Cost of Debt
interest_expense = abs(income_statement.T['Interest Expense'][0])
rd1 = interest_expense / d1

# Corporate Tax
tc = income_statement.T['Income Tax Expense'][0] / income_statement.T['Income Before Tax'][0]

wacc = (e1 / v * re1) + (d1 / v * rd1 * (1 - tc))

print('Weighted Average Cost of Capital for {} is {}%'.format(ticker, round(wacc,2)))

## Return on Invested Capital

In [ ]:
income_statement = stock.get_financials(freq='quarterly')
income_statement = income_statement.fillna(0)

invested_capital = (ebalance_sheet.T['Invested Capital'][0] + ebalance_sheet.T['Invested Capital'][1]) / 2

operating_income = income_statement.T['Operating Income'].sum()

# Net Operating Profit After Tax
nopat = operating_income * (1 - tc)

roic = nopat / invested_capital * 100

print('Return on Invested Capital for {} is {}%'.format(ticker, round(roic,2)))

## WACC vs ROIC
ROIC > WACC = Company generates higher roi than the company to raise the capital needed for that investment

In [ ]:
wacc_roic = wacc + roic
wacc_portion = round(wacc / wacc_roic * 100, 2)
roic_portion = round(roic / wacc_roic * 100, 2)

wacc_roic_status = None

if wacc_portion * 3 < roic_portion:
    wacc_roic_status = 'the company invests in profitable investment.'
else:
    wacc_roic_status = 'the company earns lower roi on its investment than the cost of funding the projects'

print('WACC accomodate {}% vs ROIC accomodate {}%, hence {}'.format(wacc_portion, roic_portion, wacc_roic_status))

## Peter Lynch Fair Value

In [ ]:
income_statement = stock.get_financials(freq='quarterly')
income_statement = income_statement.fillna(0)

peg_ratio = stock.info['pegRatio']

all_net_income = sum(income_statement.T['Net Income'])
no_shares_outstanding = get_balance_sheet_from_yfinance_web(ticker).T['Ordinary Shares Number'][0]
eps = all_net_income / no_shares_outstanding

income_statement = stock.get_financials(freq='yearly')
income_statement = income_statement.fillna(0)

all_ebit = income_statement.T['Ebit']
all_depreciation = cash_flow_statement.T['Depreciation']
all_ebitda = all_ebit + all_depreciation
all_ebitda = all_ebitda.iloc[::-1]
all_ebitda_growth = all_ebitda.pct_change().dropna()
average_ebitda_growth = all_ebitda_growth.sum() / all_ebitda_growth.shape[0] * 100

peter_lynch_fair_value = peg_ratio * average_ebitda_growth * eps

print('Peter Lynch Fair Value for {}% is {}'.format(ticker, round(peter_lynch_fair_value,2)))

## Intrinsic Value with Discounted Free Cash Flow
The discounted free cash flow (dfcf) model - A valuation method used to estimate the value based on its expected future cash flows. In short, using eps growth to estimate future operating cash flow, uses beta as a metrics to leverage the discount rate for the reason of cash worth lesser in future due to inflation. Multiply the future operating cash flow by discount rate to get discount value for the business. Divide the number with total number of shares outstanding. Addtionally, cash and short term investment and short term and long term debt on balance sheet are uses to calculate cash per share or debt per share respectively if the business get liquidates. At last, adding cash per share and subtracting debt per share to retrieve the final intrinsic value.

In [ ]:
# cashflowstatement = stock.get_cashflow() # used last fiscal year
# operating_cash_flow = cashflowstatement.T['Total Cash From Operating Activities'][0]
operating_cash_flow = stock.info['operatingCashflow'] # used ttm 

# balance_sheet = stock.get_balancesheet(freq='quarterly')
# cash_and_short_term_investment = balance_sheet.T['Cash'][0] + balance_sheet.T['Short Term Investments'][0]
cash_and_short_term_investment = stock.info['totalCash']
# short_term_and_long_term_debt = balance_sheet.T['Total Current Liabilities'][0] + balance_sheet.T['Long Term Debt'][0]
short_term_and_long_term_debt = stock.info['totalDebt']

last_closing_price = stock.info['previousClose']

# no_shares_outstanding = stock.info['sharesOutstanding']
no_shares_outstanding = get_balance_sheet_from_yfinance_web(ticker).T['Ordinary Shares Number'][0]

current_year = datetime.date.today().year

# Alter yourself
eps_growth_1_5y = float(input()) / 100
eps_growth_6_10y = 10 / 100
eps_growth_11_20y = 4.18 / 100
# Creates a list of eps growth 
eps_growth_list = [ [eps_growth_1_5y] * 5 + [eps_growth_6_10y] * 5 + [eps_growth_11_20y] * 10 ]
# Flattens the list of list
eps_growth_list = [element for sublist in eps_growth_list for element in sublist]

beta = stock.info['beta']
discount_rate = 0

# Identify the discount rate with volatility(beta) of the stock
if beta < 0.8:
    discount_rate = 4.6 / 100
elif beta < 1:
    discount_rate = 5.6 / 100
elif beta < 1.1:
    discount_rate = 6.1 / 100
elif beta < 1.2:
    discount_rate = 6.6 / 100
elif beta < 1.3:
    discount_rate = 7.1 / 100
elif beta < 1.4:
    discount_rate = 7.6 / 100
elif beta < 1.5:
    discount_rate = 8.1 / 100
else:
    discount_rate = 8.6 / 100
    
# print('Operating Cash Flow: ', operating_cash_flow)
# print('Cash and Short Term Investment: ', cash_and_short_term_investment)
# print('Short Term and Long Term Debt: ', short_term_and_long_term_debt)
# print('No. Shares Outstanding: ', no_shares_outstanding)
# print('Discount Rate: ', discount_rate)

# Initialize all parameters due to some different equation afterward
initial_projected_year = current_year + 1
initial_projected_operating_cash_flow = operating_cash_flow * (1 + eps_growth_list[0])
initial_discount_factor = 1 / (1 + discount_rate)
initial_discounted_value = ( initial_projected_operating_cash_flow * initial_discount_factor )

# Appends all initialized parameters to the list for later access
projected_year_list = [initial_projected_year]
projected_operating_cash_flow_list = [initial_projected_operating_cash_flow]
discount_factor_list = [initial_discount_factor]
discounted_value_list = [initial_discounted_value]

# A 20 years discounted cash flow calculator
for idx, projected_year in enumerate(range(1, 20)):
    
    year = projected_year_list[-1] + 1
    projected_year_list.append(year)
    
    projected_operating_cash_flow = projected_operating_cash_flow_list[-1] * (1 + eps_growth_list[1 + idx])
    projected_operating_cash_flow_list.append(projected_operating_cash_flow)
    
    discount_factor = discount_factor_list[-1] / (1 + discount_rate)
    discount_factor_list.append(discount_factor)
    
    discounted_value = projected_operating_cash_flow * discount_factor
    discounted_value_list.append(discounted_value)

# Merge into pandas dataframe
projected_df = pd.DataFrame({'Projected Year': projected_year_list,
                             'Projected Operating Cash Flow': projected_operating_cash_flow_list,
                             'Discount Factor': discount_factor_list,
                             'Discounted Value': discounted_value_list})

display(projected_df)

total_projected_cash_flow = round( sum(discounted_value_list), 2)

intrinsic_value_before_cash_debt = round( total_projected_cash_flow / no_shares_outstanding, 2)

cash_per_share = round( cash_and_short_term_investment / no_shares_outstanding, 2)

debt_per_share = round( short_term_and_long_term_debt / no_shares_outstanding, 2)

intrinsic_value_per_share = round( intrinsic_value_before_cash_debt + cash_per_share - debt_per_share, 2)

discount_or_premium = round( (last_closing_price - intrinsic_value_per_share) / intrinsic_value_per_share * 100, 2)

print('Total Projected Cash Flow: ', total_projected_cash_flow)
print('Intrinsic Value Before Cash & Debt: ', intrinsic_value_before_cash_debt)
print('Cash Per Share: ', cash_per_share)
print('Debt Per Share: ', debt_per_share)
print('Intrinsic Value Per Share: ', intrinsic_value_per_share)
print('Currently (Discount)/Premium: {}%'.format(discount_or_premium))

## Margin of Safety

In [ ]:
safety_rate = 20 / 100
bargain_price = round(intrinsic_value_per_share * (1 - safety_rate), 2)
percent_undervalue = round((last_closing_price - bargain_price) / bargain_price * 100, 2)
print('Bargain Price for {} is at {}, currently {}, {}% (safer)/riskier'.format(ticker, bargain_price, last_closing_price, percent_undervalue))

## Historical Price

In [ ]:
df = stock.history(period='20y', actions=False)['Close'].to_frame()
df['MA50'] = df['Close'].rolling(window=50, min_periods=1).mean()
df['MA150'] = df['Close'].rolling(window=150, min_periods=1).mean()
df['MA200'] = df['Close'].rolling(window=200, min_periods=1).mean()
fig, ax = plt.subplots(figsize=(12, 6), dpi=150)
ax.grid()
ax.plot(df.index, df.Close, label='Close Price')
ax.plot(df.index, df.MA50, label='50-day MA')
ax.plot(df.index, df.MA150, label='150-day MA')
ax.plot(df.index, df.MA200, label='200-day MA')
ax.set_xlabel('Date')
ax.set_ylabel('Price ($)')
ax.set_title('{} 5-Year History Price'.format(ticker))
ax.legend()
plt.show()

## SMA Signal

In [ ]:
df = stock.history(period='5y', actions=False)['Close'].to_frame()
df['MA50'] = df['Close'].rolling(window=50, min_periods=1).mean()
df['MA150'] = df['Close'].rolling(window=150, min_periods=1).mean()
df['MA200'] = df['Close'].rolling(window=200, min_periods=1).mean()
df['signal'] = 0.0
df['signal'] = np.where(df['MA50'] > df['MA200'], 1.0, 0.0)
df['position'] = df['signal'].diff()

fig, ax = plt.subplots(figsize=(12, 6), dpi=150)
ax.grid()
ax.plot(df.index, df.Close, label='Close Price')
ax.plot(df.index, df.MA50, label='50-day MA')
ax.plot(df.index, df.MA150, label='150-day MA')
ax.plot(df.index, df.MA200, label='200-day MA')
ax.plot(df[df['position'] == 1].index, df['MA50'][df['position'] == 1], '^', markersize=10, color='g', label='buy')
ax.plot(df[df['position'] == -1].index, df['MA50'][df['position'] == -1], 'v', markersize=10, color='r', label='sell')
ax.set_xlabel('Date')
ax.set_ylabel('Price ($)')
ax.set_title('{} Buying and Selling Signal'.format(ticker))
ax.legend()
plt.show()